In [1]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import warnings
import re

import os
import requests
from PIL import Image
from io import BytesIO
from urllib.parse import urlparse
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

In [68]:
# Instantiate the data required in lists
users_list = ['ur62522856']
# ur22171966 1910 ratings
# ur46592925 4 ratings
# ur62522856 338 ratings
user_id = []
title_list = []
description_list = []
img_list = []
year_list = []
director_list = []
star_list = []
duration_list = []
advisory_list = []
genre_list = []
vote_list = []
movie_rating_list = []
user_rating_list = []
error_msg = []

In [67]:
for user in range(len(users_list)):
    print("Scraping for User {}".format(user+1))
    driver = webdriver.Chrome(ChromeDriverManager().install())
    url = 'https://www.imdb.com/user/{}/ratings'.format(users_list[user])
    time.sleep(1)
    driver.get(url)
    time.sleep(1)
    
    sel = Selector(text = driver.page_source)
    num_of_ratings = sel.css(".lister-list-length span::text").extract_first().replace(',','').split(' ')[0]
    rating_pages = int(int(num_of_ratings)/100) + 1  
    user_id += [users_list[user] for i in range(int(num_of_ratings))]
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
#     #scroll
#     scroll_height = driver.execute_script("return document.body.scrollHeight")
    
#     # Set scroll step and delay
#     scroll_step = 700 
#     scroll_delay = 1  

#     current_scroll_position = 0

    for x in range(rating_pages):
        sel = Selector(text = driver.page_source)
        reviews = driver.find_elements(By.CSS_SELECTOR, 'div.lister-item.mode-detail')
        
        # filter out the images that have a title attribute with the value "list image"
        ##Get img url
        image_tags = soup.find_all('img', {'class': 'loadlate', 'title': lambda value: value is None or value != 'list image'})
        for img_tag in image_tags:
            # Get the URL of the movie poster image
            img_url = img_tag['loadlate']
#             print(img_url)
            img_list.append(img_url)
        
#         current_scroll_position = 0
#         scroll_height = driver.execute_script("return document.body.scrollHeight")
#         while current_scroll_position < scroll_height:
#             driver.execute_script(f"window.scrollTo(0, {current_scroll_position + scroll_step});")
#             current_scroll_position += scroll_step
#             time.sleep(scroll_delay)
#             new_scroll_height = driver.execute_script("return document.body.scrollHeight")
#             if new_scroll_height > scroll_height:
#                 scroll_height = new_scroll_height
#             if current_scroll_position >= scroll_height:
#                 break
        
        for review in tqdm(reviews):
            try:
                sel2 = Selector(text = review.get_attribute('innerHTML'))
                
                ##Get img path
#                 try:
#                     img_path = sel2.css('.lister-item-image.ribbonize img::attr(src)').extract_first().strip()
# #                     print(img_path)
#                 except:
#                     img_path = np.NaN
                
                ## Get movie title
                try:
                    title = sel2.css('.lister-item-content a::text').extract_first().strip()
                    episode = sel2.css('.lister-item-content a::text').getall()[1].strip()
                    if episode != "":
                        title += (" - " + episode)
#                     print(title)
                except:
                    title= np.NaN
                    
                ## Get movie description
                try:
                    advisory = sel2.css('.certificate::text').extract_first()
                    duration = sel2.css('.runtime::text').extract_first()
                    description = None
                    if advisory == None and duration == None:
                        description = sel2.css('p::text').getall()[3].strip()
                    elif advisory == None or duration == None:
                        description = sel2.css('p::text').getall()[5].strip()
                    else:
                        description = sel2.css('p::text').getall()[7].strip()
                except:
                    description = np.NaN
                ## Get movie year
                try:
                    year = sel2.css('.lister-item-year.text-muted.unbold::text').extract_first().strip().replace('(','').replace(')','')
                    year = re.sub(r'[a-zA-Z\s]+', '', year)
                except:
                    year = np.NaN
                ## Get directors and staff
                try:
                    staff = sel2.css('.text-muted a::text').getall()
                    text = sel2.css('.text-muted.text-small::text').getall()
                    text2 = [x.strip() for x in text]
                    commas = text2.count(',')
                    stars_index = text2.index("Stars:")
                    count = 0
                    for i in range(stars_index, len(text2)-1):
                        if text2[i] == ',':
                            count+=1
                    stars = staff[-(count+1):]
                    # if directors are recorded
                    if "Director:" in text2 or "Directors:" in text2:
                        directors = staff[:(commas-count)+1]
                    else:
                        directors = ""
                except:
                    stars = np.NaN
                    directors = np.NaN
                ## Get movie duration
                try:
                    duration = duration.strip()
                except:
                    duration = np.NaN
                ## Get viewer advisory
                try:
                    advisory = advisory.strip()
                except:
                    advisory = np.NaN
                ## Get Genre
                try:
                    genre = sel2.css('.genre::text').extract_first().strip()
                except:
                    genre = np.NaN
                ## Get votes
                try:
                    votes = sel2.css('.text-muted.text-small span::text').getall()[-1]
                    votes = int(votes.replace(',','').split(' ')[0])
                except:
                    votes = np.NaN
                ## Get movie rating
                try:
                    movie_rating = sel2.css('.ipl-rating-star__rating::text').getall()[0]
                    movie_rating = float(movie_rating.replace(',','').split(' ')[0])
                except:
                    movie_rating = np.NaN
                ## Get user rating
                try:
                    user_rating = sel2.css('.ipl-rating-star__rating::text').getall()[1]
                    user_rating = int(user_rating.replace(',','').split(' ')[0])
                except:
                    user_rating = np.NaN
                
                title_list.append(title)
#                 img_list.append(img_path)
                description_list.append(description)
                year_list.append(year)
                director_list.append(directors)
                star_list.append(stars)
                duration_list.append(duration)
                advisory_list.append(advisory)
                genre_list.append(genre)
                vote_list.append(votes)
                movie_rating_list.append(movie_rating)
                user_rating_list.append(user_rating)
                
            except Exception as e:
                error_msg.append(e)
        
        try:
            next_page_url = sel.css("a.flat-button.lister-page-next.next-page::attr(href)").extract_first()
            full_next_page_url = "https://www.imdb.com" + next_page_url
            driver.get(full_next_page_url)
            response = requests.get(full_next_page_url)
            soup = BeautifulSoup(response.content, 'html.parser')
        except:
            print("No more pages to browse")

Scraping for User 1
https://m.media-amazon.com/images/M/MV5BMDE5ZjAwY2YtOWM5Yi00ZWNlLWE5ODQtYjA4NzA1NGFkZDU5XkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMjIyNDg4OTYwOF5BMl5BanBnXkFtZTgwOTQ3ODgwOTE@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTY5NjI2MjQxMl5BMl5BanBnXkFtZTgwMDA2MzM2NzE@._V1_UY209_CR1,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BNzgzODA5MTU3MF5BMl5BanBnXkFtZTcwODY4MDEwNg@@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTcyODc4Njg4OF5BMl5BanBnXkFtZTgwNDIwNDA4MTE@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BN2RkZjk0ODctNWE0MS00MjFlLTliMDMtZTFhMWM2NTAxMDg1XkEyXkFqcGdeQXVyNjc0ODMyMDc@._V1_UY209_CR1,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMDNiZjIzMzYtMDg1Zi00ZjM3LWFlZjUtZmNhMTQ1MDU3ODU0XkEyXkFqcGdeQXVyNjc3OTE4Nzk@._V1_UY209_CR6,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BODZkZjUxNmEtMGEyOS00ZDY5L

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 29.21it/s]


https://m.media-amazon.com/images/M/MV5BMTY3MjM1Mzc4N15BMl5BanBnXkFtZTgwODM0NzAxMDE@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BOWQ1MDE1NzgtNTQ4OC00ZjliLTllZDAtN2IyOTVmMTc5YjUxXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BODkxNGQ1NWYtNzg0Ny00Yjg3LThmZTItMjE2YjhmZTQ0ODY5XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTk0MDQ3MzAzOV5BMl5BanBnXkFtZTgwNzU1NzE3MjE@._V1_UY209_CR1,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BZmExNmEwYWItYmQzOS00YjA5LTk2MjktZjEyZDE1Y2QxNjA1XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTUwODU3NjQxNF5BMl5BanBnXkFtZTgwODE2NTE4NTE@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BZDA0OGQxNTItMDZkMC00N2UyLTg3MzMtYTJmNjg3Nzk5MzRiXkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_UY209_CR1,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BNDg1NTU2OWEtM

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 25.51it/s]


https://m.media-amazon.com/images/M/MV5BZTJhN2FkYWEtMGI0My00YWM4LWI2MjAtM2UwNjY4MTI2ZTQyXkEyXkFqcGdeQXVyNjc3MjQzNTI@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BZTI3NmJmYTQtNDg4NS00M2VlLTgzZDAtZWIwZDcyOWY5YzIzXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTkyNDY1ODQwNV5BMl5BanBnXkFtZTgwNzA2MjUwNzE@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTYwMTA4MzgyNF5BMl5BanBnXkFtZTgwMjEyMjE0MDE@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BNjg3ODQyNTIyN15BMl5BanBnXkFtZTcwMjUzNzM5NQ@@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BOGQzZTBjMjQtOTVmMS00NGE5LWEyYmMtOGQ1ZGZjNmRkYjFhXkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BOGUxOWQ4MzAtMmJjYS00M2U5LWEwZTAtYTc1YmZhNjg2NDRlXkEyXkFqcGdeQXVyMTYzMDM0NTU@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMjExOTQ4MDMyM

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 25.83it/s]


https://m.media-amazon.com/images/M/MV5BZmU0M2Y1OGUtZjIxNi00ZjBkLTg1MjgtOWIyNThiZWIwYjRiXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX140_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMDliMmNhNDEtODUyOS00MjNlLTgxODEtN2U3NzIxMGVkZTA1L2ltYWdlXkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMTI1MTY2OTIxNV5BMl5BanBnXkFtZTYwNjQ4NjY3._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BMGU2NzRmZjUtOGUxYS00ZjdjLWEwZWItY2NlM2JhNjkxNTFmXkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BOTA5NDZlZGUtMjAxOS00YTRkLTkwYmMtYWQ0NWEwZDZiNjEzXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UY209_CR0,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BNTU2ODkyY2MtMjU1NC00NjE1LWEzYjgtMWQ3MzRhMTE0NDc0XkEyXkFqcGdeQXVyMjM4MzQ4OTQ@._V1_UY209_CR1,0,140,209_AL_.jpg
https://m.media-amazon.com/images/M/MV5BZjdkOTU3MDktN2IxOS00OGEyLWFmMjktY2FiMmZkNWIyODZiXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UY209_CR0,0,1

100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [00:01<00:00, 27.22it/s]

No more pages to browse


In [63]:
# Storing all data in dataframe
rating_df = pd.DataFrame({
     "UserID": user_id,
     "Title":title_list,
     "Img_Path": img_list,
     "Year":year_list,
     "Description":description_list,
     "Directors":director_list,
     "Stars": star_list,
     "Viewer_Advisory": advisory_list,
     "Duration": duration_list,
     "Genre": genre_list,
     "Votes": vote_list, 
     "Movie_Rating": movie_rating_list,
     "User_Rating": user_rating_list})

In [64]:
rating_df

,UserID,Title,Img_Path,Year,Description,Directors,Stars,Viewer_Advisory,Duration,Genre,Votes,Movie_Rating,User_Rating
0,ur62522856,The Killing,https://m.media-amazon.com/images/M/MV5BMDE5Zj...,1956,Crook Johnny Clay assembles a five-man team to...,[Stanley Kubrick],"[Sterling Hayden, Coleen Gray, Vince Edwards, ...",Approved,1 hr 24 min,"Crime, Drama, Film-Noir",91819,7.9,7
1,ur62522856,Indignation,https://m.media-amazon.com/images/M/MV5BMjIyND...,2016,"In 1951, Marcus, a working-class Jewish studen...",[James Schamus],"[Logan Lerman, Sarah Gadon, Tijuana Ricks, Sue...",R,1 hr 50 min,"Drama, Romance",12342,6.8,5
2,ur62522856,Only Yesterday,https://m.media-amazon.com/images/M/MV5BMTY5Nj...,1991,A twenty-seven-year-old office worker travels ...,[Isao Takahata],"[Miki Imai, Toshirô Yanagiba, Yoko Honna, Mayu...",PG,1 hr 59 min,"Animation, Drama, Romance",33847,7.6,9
3,ur62522856,Take Shelter,https://m.media-amazon.com/images/M/MV5BNzgzOD...,2011,"Plagued by a series of apocalyptic visions, a ...",[Jeff Nichols],"[Michael Shannon, Jessica Chastain, Shea Whigh...",R,2 hr,"Drama, Horror, Sci-Fi",103012,7.3,8
4,ur62522856,What If,https://m.media-amazon.com/images/M/MV5BMTcyOD...,2013,"Wallace, who is burned out from a string of fa...",[Michael Dowse],"[Daniel Radcliffe, Zoe Kazan, Megan Park, Adam...",PG-13,1 hr 38 min,"Comedy, Romance",79395,6.8,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,ur62522856,Family Guy,https://m.media-amazon.com/images/M/MV5BODEwZj...,1999–,"In a wacky Rhode Island town, a dysfunctional ...",,"[Seth MacFarlane, Alex Borstein, Seth Green, M...",TV-MA,22 min,"Animation, Comedy",347533,8.2,8
334,ur62522856,South Park,https://m.media-amazon.com/images/M/MV5BZjNhOD...,1997–,Follows the misadventures of four irreverent g...,,"[Trey Parker, Matt Stone, Isaac Hayes, Mona Ma...",TV-14,22 min,"Animation, Comedy",380832,8.7,10
335,ur62522856,The Simpsons,https://m.media-amazon.com/images/M/MV5BYjFkMT...,1989–,The satiric adventures of a working-class fami...,,"[Dan Castellaneta, Nancy Cartwright, Harry She...",TV-14,22 min,"Animation, Comedy",414335,8.7,9
336,ur62522856,American Sniper,https://m.media-amazon.com/images/M/MV5BMTkxNz...,2014,Navy S.E.A.L. sniper Chris Kyle's pinpoint acc...,[Clint Eastwood],"[Bradley Cooper, Sienna Miller, Kyle Gallner, ...",R,2 hr 13 min,"Action, Biography, Drama",497784,7.3,7


In [125]:
# Export dataset
rating_df.to_csv(path_or_buf = "users_ratings.csv"
                          , index = False,  encoding='utf-8-sig')

In [65]:
##image downloading
def download_image(url, save_path):
    try:
        response = requests.get(url)
        response.raise_for_status()

        img = Image.open(BytesIO(response.content))


        img_save_path = save_path
        img.save(img_save_path, format="JPEG")

        print(f"Image saved at {img_save_path}")
    except Exception as e:
        print(f"Error downloading image from {url}: {e}")

# Folder for downloading
img_folder = "downloaded_images_ratings"
os.makedirs(img_folder, exist_ok=True)

for idx, row in rating_df.iterrows():
    user_id = row['UserID']
    title = row['Title']
    img_url = row['Img_Path']
    
    title = re.sub(r'[^\w\s!-]|[.!?]', '', title)

    # File extension name
    img_filename = f"{user_id}_{title}.jpg"
    img_save_path = os.path.join(img_folder, img_filename)

    # Download and save the image
    download_image(img_url, img_save_path)
    
    #See image in python
    #img = Image.open(img_save_path)
    #plt.imshow(img)
    #plt.savefig(img_save_path)
    #plt.show()

Image saved at downloaded_images_ratings\ur62522856_The Killing.jpg
Image saved at downloaded_images_ratings\ur62522856_Indignation.jpg
Image saved at downloaded_images_ratings\ur62522856_Only Yesterday.jpg
Image saved at downloaded_images_ratings\ur62522856_Take Shelter.jpg
Image saved at downloaded_images_ratings\ur62522856_What If.jpg
Image saved at downloaded_images_ratings\ur62522856_Angel Beats.jpg
Image saved at downloaded_images_ratings\ur62522856_FateZero.jpg
Image saved at downloaded_images_ratings\ur62522856_Neon Genesis Evangelion.jpg
Image saved at downloaded_images_ratings\ur62522856_Naruto.jpg
Image saved at downloaded_images_ratings\ur62522856_The Founder.jpg
Image saved at downloaded_images_ratings\ur62522856_The Red Turtle.jpg
Image saved at downloaded_images_ratings\ur62522856_Free Fire.jpg
Image saved at downloaded_images_ratings\ur62522856_War Dogs.jpg
Image saved at downloaded_images_ratings\ur62522856_Manchester by the Sea.jpg
Image saved at downloaded_images_rat

Image saved at downloaded_images_ratings\ur62522856_Pans Labyrinth.jpg
Image saved at downloaded_images_ratings\ur62522856_Paper Towns.jpg
Image saved at downloaded_images_ratings\ur62522856_The Incredibles.jpg
Image saved at downloaded_images_ratings\ur62522856_Spotlight.jpg
Image saved at downloaded_images_ratings\ur62522856_I Am Legend.jpg
Image saved at downloaded_images_ratings\ur62522856_Kiss Kiss Bang Bang.jpg
Image saved at downloaded_images_ratings\ur62522856_12 Years a Slave.jpg
Image saved at downloaded_images_ratings\ur62522856_Adaptation.jpg
Image saved at downloaded_images_ratings\ur62522856_Indiana Jones and the Kingdom of the Crystal Skull.jpg
Image saved at downloaded_images_ratings\ur62522856_The Boxtrolls.jpg
Image saved at downloaded_images_ratings\ur62522856_The Croods.jpg
Image saved at downloaded_images_ratings\ur62522856_Two and a Half Men.jpg
Image saved at downloaded_images_ratings\ur62522856_Jaws.jpg
Image saved at downloaded_images_ratings\ur62522856_Her.jpg

Image saved at downloaded_images_ratings\ur62522856_Requiem for a Dream.jpg
Image saved at downloaded_images_ratings\ur62522856_Memento.jpg
Image saved at downloaded_images_ratings\ur62522856_Léon The Professional.jpg
Image saved at downloaded_images_ratings\ur62522856_Nightcrawler.jpg
Image saved at downloaded_images_ratings\ur62522856_Elysium.jpg
Image saved at downloaded_images_ratings\ur62522856_Birdman or The Unexpected Virtue of Ignorance.jpg
Image saved at downloaded_images_ratings\ur62522856_Goosebumps.jpg
Image saved at downloaded_images_ratings\ur62522856_Prisoners.jpg
Image saved at downloaded_images_ratings\ur62522856_Up in the Air.jpg
Image saved at downloaded_images_ratings\ur62522856_No Country for Old Men.jpg
Image saved at downloaded_images_ratings\ur62522856_Warrior.jpg
Image saved at downloaded_images_ratings\ur62522856_Dogma.jpg
Image saved at downloaded_images_ratings\ur62522856_30 Minutes or Less.jpg
Image saved at downloaded_images_ratings\ur62522856_Blade Runner